In [ ]:
import sys
import torch
from models import create_model
from options.test_options import TestOptions

# Path to your pre-trained weights
weights_path = "/Users/yahyarahhawi/Developer/Film/15_net_G_A.pth"

# Override sys.argv to inject minimal CLI args for TestOptions
original_argv = sys.argv
sys.argv = [
    original_argv[0],
    '--dataroot', 'dummy',        # Needed to satisfy required '--dataroot' argument
    '--model', 'cycle_gan',       # Specifies the CycleGAN model
    '--dataset_mode', 'single',   # We are feeding images manually
    '--gpu_ids', '-1',            # Force CPU usage (no CUDA)
]

# Parse the options
opt = TestOptions().parse()

# Restore sys.argv to its original state
sys.argv = original_argv

# Set additional options manually
opt.isTrain = False  # Indicates we're running in evaluation/test mode
opt.no_dropout = True  # No dropout for inference
opt.batch_size = 1  # Inference only works with batch size = 1
opt.load_size = 256  # Resize to 256x256 during preprocessing (adjust as needed)
opt.crop_size = 256  # Crop size must match load size for this example

# Create the CycleGAN model
cycleGAN = create_model(opt)
cycleGAN.eval()  # Set the model to evaluation mode

# Manually load netG_A weights
state_dict = torch.load(weights_path, map_location="cpu")
cycleGAN.netG_A.load_state_dict(state_dict)

# Access the generator (netG_A)
model_netG_A = cycleGAN.netG_A

# Print a summary to verify everything is working
print("CycleGAN model created successfully!")
print(f"Generator loaded: {model_netG_A}")

In [ ]:
import torch
import os

def export_cycleGAN_to_onnx(
    model_netG_A: torch.nn.Module,
    onnx_path: str = "cycleGAN_genA.onnx",
    input_size: int = 256
):
    """
    Exports the given netG_A generator to ONNX format.

    :param model_netG_A: The PyTorch nn.Module for netG_A (e.g., cycleGAN.netG_A).
    :param onnx_path:    Where to save the ONNX file.
    :param input_size:   The (height, width) for the input image.
    """
    # Put the model in eval mode
    model_netG_A.eval()

    # Create a dummy input of shape [batch_size=1, channels=3, height=input_size, width=input_size]
    dummy_input = torch.randn(1, 3, input_size, input_size, device="cpu")

    # Export to ONNX
    torch.onnx.export(
        model_netG_A,
        dummy_input,
        onnx_path,
        input_names=["input"],
        output_names=["output"],
        opset_version=11,  # Adjust opset version if needed (11 is widely compatible)
        do_constant_folding=True
    )

    print(f"ONNX model saved to {onnx_path}")


# Export the loaded generator (netG_A) to ONNX
onnx_output_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN_genA.onnx"
export_cycleGAN_to_onnx(model_netG_A=cycleGAN.netG_A, onnx_path=onnx_output_path, input_size=256)

In [ ]:
import torch

# Create a dummy input with the same shape as your input images
dummy_input = torch.randn(1, 3, 256, 256)

# Trace the model to produce a TorchScript version
traced_model = torch.jit.trace(model_netG_A, dummy_input)

# Save the TorchScript model (optional, for debugging)
traced_model_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN_traced.pt"
traced_model.save(traced_model_path)

print(f"TorchScript model saved to {traced_model_path}")

In [ ]:
import coremltools as ct

# Convert the TorchScript model to Core ML
coreml_model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input", shape=(1, 3, 256, 256))],
    minimum_deployment_target=ct.target.iOS13  # Specify deployment target if needed
)

# Save the Core ML model with the correct extension
coreml_model_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN.mlpackage"
coreml_model.save(coreml_model_path)

print(f"Core ML model saved to {coreml_model_path}")

*Test Core ML*

In [ ]:
import coremltools as ct
from PIL import Image
import numpy as np

# Load the Core ML model
coreml_model_path = "/Users/yahyarahhawi/Developer/Film/cycleGAN.mlpackage"
model = ct.models.MLModel(coreml_model_path)

# Load an input image
input_image_path = "/Users/yahyarahhawi/Downloads/epoch017_real_A (1).png"
image = Image.open(input_image_path).convert("RGB")
image = image.resize((256, 256))  # Resize to the expected input size

# Perform inference
output = model.predict({"input": image})

# Access the output using the correct key
output_image_data = output["var_309"]  # Use the correct key

# Process the output: Convert from NumPy array to PIL Image
output_image_data = (output_image_data.squeeze().transpose(1, 2, 0) * 255).clip(0, 255).astype(np.uint8)  # (H, W, C)
output_image = Image.fromarray(output_image_data)

# Save or display the output image
output_image.save("/Users/yahyarahhawi/Developer/Film/output_image.jpg")
output_image.show()